In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [2]:
df_train = pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/train.parquet")

In [3]:
Na = df_train.isna().sum()*100/(df_train.shape[0])


In [4]:
Na_cols = Na.sort_values(ascending = False).head(25).index

In [5]:
df_train.drop(columns = Na_cols, inplace = True)

In [6]:
del Na

In [7]:
cat_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [8]:
y = pd.read_csv("../input/amex-default-prediction/train_labels.csv")

In [9]:
y.head()

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0


In [10]:
df_train = pd.merge(df_train, y, on = "customer_ID")

In [11]:
df_train = df_train.groupby("customer_ID").head(2)

In [12]:
df_train = df_train.fillna(method = "ffill")

In [13]:
df_date = df_train.pop("S_2")
del df_date

In [14]:
df_train = df_train.set_index("customer_ID", drop = True).sort_index()

In [15]:
y = df_train.pop("target")

In [16]:
features = list(set(df_train.columns) - set(cat_col)) 

In [17]:
X = df_train[features]

In [18]:
del df_train

In [19]:
#model = LogisticRegression()
#model.fit(X,y)
#model.score(X,y)

In [20]:
#import statsmodels.api as sm
#model = sm.Logit(y, X).fit()
#print(model.summary())

In [21]:
nonsig_cols = ["D_65", "S_23", "D_135", "S_24", "B_26", "D_143", "B_40", "D_141", "B_19",
              "S_22", "D_137", "R_7", "D_71", "B_2", "B_27", "D_69", "R_20", "D_83", "D_75",
              "B_36", "R_25", "D_139", "R_28", "R_10", "R_24", "D_119", "R_4", "B_10"]

In [22]:
sig_cols = list(set(X.columns) - set(nonsig_cols))

In [23]:
X = X[sig_cols]
model = LogisticRegression()
model.fit(X,y)
model.score(X,y)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.8137658786071309

In [24]:
del X
del y

In [25]:
test_df = (pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/test.parquet")
            .groupby("customer_ID")
           .tail(1)
           .fillna(method = "ffill")
           .set_index("customer_ID", drop = True)
           .sort_index())

In [26]:
test_df = test_df[sig_cols]

In [27]:
y_pred = model.predict_proba(test_df)

In [28]:
del test_df

In [29]:
#y_pred

In [30]:
sub = pd.read_csv("../input/amex-default-prediction/sample_submission.csv")

In [31]:
sub = sub.sort_values("customer_ID")

In [32]:
sub["prediction"] = y_pred[:, 1]

In [33]:
sub.to_csv("submission.csv", index = False)